This is a Notebook to test the ADAM batch OPM API.<br>
It was taken from test.py

In [ ]:
from adam import Auth
from adam import Batch
from adam import Projects
from adam import RestRequests
from adam import AuthorizingRestProxy
import time
import os

In [ ]:
rest = RestRequests("http://pro-equinox-162418.appspot.com/_ah/api/adam/v1")
auth = Auth()
auth.set_rest_accessor(rest)

tokenFile = os.getcwd() + '/token.txt'
# Opening with "a+" instead of "r" creates the file if it doesn't exist.
with open(tokenFile, "a+") as f:
    f.seek(0)
    token = f.read()

try:
    if not auth.authorize(token):
        if auth.initial_authorization():
            with open(tokenFile, "w") as f:
                f.write(auth.get_token())
except RuntimeError as e:
    print('Encountered server error while attempting to authorize: ' + str(e))

if auth.get_token() == "":
    print('Could not authorize user.')
else:
    print('Welcome, ' + auth.get_user())
    

In [ ]:
# If non-empty, auth.get_token() can now be used to authorize calls to other API methods.
auth_rest = AuthorizingRestProxy(rest, auth.get_token())

projects = Projects(auth_rest)
project = projects.new_project('ffffffff-ffff-ffff-ffff-ffffffffffff', None, "parent")
child = projects.new_project(project.get_uuid(), None, "child")
print('Current projects, including newly-created parent and child:')
projects.print_projects()
projects.delete_project(child.get_uuid())
projects.delete_project(project.get_uuid())

In [ ]:
state_vec = [130347560.13690618,
             -74407287.6018632,
             -35247598.541470632,
             23.935241263310683,
             27.146279819258538,
             10.346605942591514]

### Initiate Batch Class and Set Params

In [ ]:
batch_run = Batch()
batch_run.set_start_time('2017-10-04T00:00:00Z')
batch_run.set_end_time('2017-10-11T00:00:00Z')
batch_run.set_state_vector('2017-10-04T00:00:00.000Z', state_vec)

In [ ]:
# Optional parameters (uncomment to use)
# batch_run.set_propagator_uuid("00000000-0000-0000-0000-000000000002")    # Only Sun as point mass, nothing else
# batch_run.set_step_size(3600, 'min')
# batch_run.set_mass(500.5)
# batch_run.set_solar_rad_area(25.2)
# batch_run.set_solar_rad_coeff(1.2)
# batch_run.set_drag_area(33.3)
# batch_run.set_drag_coeff(2.5)
# batch_run.set_originator('Robot')
# batch_run.set_object_name('TestObj')
# batch_run.set_object_id('test1234')
# batch_run.set_description('some description')

### Submit Batch

In [ ]:
print("Initial state %s" % batch_run)
print
print(batch_run.generate_opm())

In [ ]:
batch_run.submit()
print
print("Final state %s" % batch_run)

#while not batch_run.is_ready():
#    time.sleep(5)

### Get Status

In [ ]:
batch_run.is_ready()

In [ ]:
batch_run.get_calc_state()

### Get Ephemeris

In [ ]:
part_count = batch_run.get_parts_count()
print("Parts count: %s" % (part_count))

In [ ]:
part_to_get = 1
eph = batch_run.get_part_ephemeris(part_to_get)
print("Ephemeris")
print(eph)